## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint George,US,37.1041,-113.5841,77.47,30,1,1.99,clear sky
1,1,La Palma,US,33.8464,-118.0467,96.12,19,0,5.01,clear sky
2,2,Bilibino,RU,68.0546,166.4372,8.71,92,100,6.08,overcast clouds
3,3,Durres,AL,41.3231,19.4414,51.98,77,82,5.53,broken clouds
4,4,Port Macquarie,AU,-31.4333,152.9167,66.88,47,0,10.45,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 35
What is your desired maximum temperature for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Durres,AL,41.3231,19.4414,51.98,77,82,5.53,broken clouds
4,4,Port Macquarie,AU,-31.4333,152.9167,66.88,47,0,10.45,clear sky
5,5,Mataura,NZ,-46.1927,168.8643,48.20,99,100,1.05,overcast clouds
7,7,Saint-Philippe,RE,-21.3585,55.7679,74.75,83,96,10.92,overcast clouds
8,8,Pisco,PE,-13.7000,-76.2167,69.85,68,0,9.22,clear sky
9,9,Cape Town,ZA,-33.9258,18.4232,66.09,72,0,3.00,clear sky
10,10,Sotnikovskoye,RU,45.0039,43.7772,39.07,72,1,4.94,clear sky
11,11,Charters Towers,AU,-20.1000,146.2667,69.82,29,73,11.52,broken clouds
14,14,New Norfolk,AU,-42.7826,147.0587,48.92,75,100,3.00,overcast clouds
15,15,Rikitea,PF,-23.1203,-134.9692,73.27,73,44,12.19,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Durres,AL,41.3231,19.4414,51.98,77,82,5.53,broken clouds
4,4,Port Macquarie,AU,-31.4333,152.9167,66.88,47,0,10.45,clear sky
5,5,Mataura,NZ,-46.1927,168.8643,48.20,99,100,1.05,overcast clouds
7,7,Saint-Philippe,RE,-21.3585,55.7679,74.75,83,96,10.92,overcast clouds
8,8,Pisco,PE,-13.7000,-76.2167,69.85,68,0,9.22,clear sky
...,...,...,...,...,...,...,...,...,...,...
659,659,Naze,JP,28.3667,129.4833,59.99,85,97,15.86,light rain
666,666,Sayyan,YE,15.1718,44.3244,50.61,27,9,4.03,clear sky
668,668,Worland,US,44.0169,-107.9554,63.75,27,38,15.01,scattered clouds
670,670,Cascais,PT,38.6979,-9.4215,62.51,65,0,3.00,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Durres,AL,51.98,broken clouds,41.3231,19.4414,
4,Port Macquarie,AU,66.88,clear sky,-31.4333,152.9167,
5,Mataura,NZ,48.20,overcast clouds,-46.1927,168.8643,
7,Saint-Philippe,RE,74.75,overcast clouds,-21.3585,55.7679,
8,Pisco,PE,69.85,clear sky,-13.7000,-76.2167,
9,Cape Town,ZA,66.09,clear sky,-33.9258,18.4232,
10,Sotnikovskoye,RU,39.07,clear sky,45.0039,43.7772,
11,Charters Towers,AU,69.82,broken clouds,-20.1000,146.2667,
14,New Norfolk,AU,48.92,overcast clouds,-42.7826,147.0587,
15,Rikitea,PF,73.27,scattered clouds,-23.1203,-134.9692,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")
        

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().count()

City                   318
Country                318
Max Temp               318
Current Description    318
Lat                    318
Lng                    318
Hotel Name             318
dtype: int64

In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Durres,AL,51.98,broken clouds,41.3231,19.4414,Palace Hotel & Spa Durrës
4,Port Macquarie,AU,66.88,clear sky,-31.4333,152.9167,Rydges Port Macquarie
5,Mataura,NZ,48.20,overcast clouds,-46.1927,168.8643,Ellie's Villa
7,Saint-Philippe,RE,74.75,overcast clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
8,Pisco,PE,69.85,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
9,Cape Town,ZA,66.09,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
10,Sotnikovskoye,RU,39.07,clear sky,45.0039,43.7772,
11,Charters Towers,AU,69.82,broken clouds,-20.1000,146.2667,Cattleman's Rest Motor Inn
14,New Norfolk,AU,48.92,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
15,Rikitea,PF,73.27,scattered clouds,-23.1203,-134.9692,People ThankYou


In [11]:
# 8a. Create the output File (CSV)
output_data_file= "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")


In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 

description=hotel_df["Current Description"]
fig= gmaps.figure(center=(30,31), zoom_level=1.5)
heat_layer= gmaps.heatmap_layer(locations, dissipating= False,
                               max_intensity=300, point_radius=4)
marker_layer= gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))